# Integrate a regression tree model to classify a person's position based on landmark distances
- Collect Training Data: Capture images with known position labels (e.g., "Standing", "Push-up Up", "Push-up Down", "Lying Down").
- Extract Features: Compute landmark distances (e.g., shoulder-to-hip, hip-to-knee, elbow angle, etc.).
- Train a Decision Tree: Use scikit-learn's DecisionTreeClassifier or DecisionTreeRegressor to classify or predict positions.
- Integrate into Real-Time Detection: Use the trained model to classify the current pose.

---
# Embeddings in a CSV file

---

In [13]:
from ACV_Classifier_Fonctions import Embeddings

emb=Embeddings()

base_folder = "./training_datasets/push-ups_images" 
features=[
    "left_shoulder_hip_dist", "left_hip_knee_dist", "left_elbow_angle",
    "right_shoulder_hip_dist", "right_hip_knee_dist", "right_elbow_angle",
    "label"
    ]
embeddings_filename = "pushup_both_sides"

pushup_both_sides_embeddings=emb.generate_embeddings(base_folder,features,embeddings_filename,show_images=False)


W0000 00:00:1743615934.039630   25698 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743615934.322230   25698 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Images processed : 29
Images skipped : 0
CSV file generated at :./embeddings/pushup_both_sides_embeddings.csv


In [16]:
features=[
    "left_shoulder_hip_dist", "left_hip_knee_dist", "left_elbow_angle",
    "label"
    ]
pushup_both_sides_embeddings.loc[:,features]

,left_shoulder_hip_dist,left_hip_knee_dist,left_elbow_angle,label
0,0.239011,0.178912,176.717493,position_up
1,0.189197,0.163043,169.883476,position_up
2,0.189197,0.163043,169.883476,position_up
3,0.144903,0.127796,179.177445,position_up
4,0.144903,0.127796,179.177445,position_up
5,0.210784,0.204381,174.219957,position_up
6,0.171415,0.176072,148.710617,position_up
7,0.078255,0.118659,92.708567,position_up
8,0.120518,0.060559,105.828327,position_up
9,0.111322,0.106019,165.656647,position_up


In [5]:
from ACV_Classifier_Fonctions import Embeddings

emb=Embeddings()

base_folder = "./training_datasets/push-ups_images" 
features=[
    "left_shoulder_hip_dist", "left_hip_knee_dist", "left_elbow_angle",
    "label"
    ]
embeddings_filename = "pushup_left_side"

pushup_left_side_embeddings=emb.generate_embeddings(base_folder,features,embeddings_filename,show_images=False)

W0000 00:00:1743615777.779998   25639 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743615777.822691   25639 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Images processed : 29
Images skipped : 0
CSV file generated at :./embeddings/pushup_left_side_embeddings.csv


In [ ]:
pushup_left_side_embeddings.head()

,left_shoulder_hip_dist,left_hip_knee_dist,left_elbow_angle,label
0,0.239011,0.178912,176.717493,position_up
1,0.189197,0.163043,169.883476,position_up
2,0.189197,0.163043,169.883476,position_up
3,0.144903,0.127796,179.177445,position_up
4,0.144903,0.127796,179.177445,position_up
5,0.210784,0.204381,174.219957,position_up
6,0.171415,0.176072,148.710617,position_up
7,0.078255,0.118659,92.708567,position_up
8,0.120518,0.060559,105.828327,position_up
9,0.111322,0.106019,165.656647,position_up


---
# Classification

---

In [1]:
from ACV_Classifier_Fonctions import Classifier
from sklearn.tree import DecisionTreeClassifier

clf=Classifier()

model=DecisionTreeClassifier()
embeddings_filename='pushup_both_sides_embeddings.csv'

clf.fit_and_save_model(model,embeddings_filename)

DecisionTreeClassifier()

In [6]:
from ACV_Classifier_Fonctions import Classifier
from sklearn.tree import DecisionTreeClassifier

clf=Classifier()

model=DecisionTreeClassifier()
embeddings_filename='pushup_left_side_embeddings.csv'

clf.fit_and_save_model(model,embeddings_filename)

DecisionTreeClassifier()

---
# Prediction

---

In [11]:
from ACV_Classifier_Fonctions import Predict

model_filename="pushup_left_side_DecisionTreeClassifier.pkl"
image_path='./training_datasets/push-ups_images/position_down/image_001.jpg'

model=Predict(model_filename)

model.predict(image_path)


W0000 00:00:1743616969.202148   26326 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743616969.257997   26328 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


array(['position_down'], dtype=object)

In [12]:
from ACV_Classifier_Fonctions import Predict

model_filename="pushup_left_side_DecisionTreeClassifier.pkl"
image_path='./training_datasets/push-ups_images/position_down/image_001.jpg'

model=Predict(model_filename)

W0000 00:00:1743616971.113750   26330 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743616971.160755   26330 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [13]:
model.predict(image_path)

array(['position_down'], dtype=object)